In [141]:
# --- IMPORT MEDIAPIPE LÊN ĐẦU TIÊN ---
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import Image as MpImage

import cv2 # Thư viện này thường gây xung đột
import numpy as np
import pandas as pd
import os
import time
from scipy.spatial import distance as dist #Sử dụng spicy để tính các CT Euclidean

In [142]:
#--- HẰNG SỐ CƠ BẢN ---
DATA_ROOT = "./Datasets_train"
OUTPUT_DIR = "./Outputs" # Output Directory
OUTPUT_CSV = os.path.join(OUTPUT_DIR, 'training-feature.csv')

In [143]:
#--- 1. Chỉ mục Mediapipe ---
RIGHT_EYE_INDICES = [33, 160, 158, 133, 153, 144]
LEFT_EYE_INDICES = [362, 385, 387, 263, 373, 380]
MOUTH_INDICES = [61, 291, 0, 17] # Chỉ cần 4 điểm chính cho MAR cơ bản

# Chỉ mục cho Pose Estimation (Yaw / Pitch / Roll) - 5 điểm
# Mũi (1), Mắt trái (33), Mắt phải (263), Miệng trái (61), Miệng phải (291)
HEAD_POSE_INDICES = [1, 33, 263, 61, 291, 152]

# Chỉ mục cho Pose Landmarker (MP Pose - 33 điểm)
NOSE_POSE_INDEX = 0
LEFT_SHOULDER_INDEX = 11
RIGHT_SHOULDER_INDEX = 12

#--- 2. Hằng số PnP (Perspective-n-Point) (Cho Head Pose) --- 
"""
Trong nhiều dự án giám sát người lái xe, CAMERA_MATRIX thường được ước tính
(ví dụ: fx = fy = width và tâm ảnh là trung tâm ảnh) vì việc calibrate (hiệu chuẩn) camera
là không thực tế.

solvePnP sử dụng CAMERA_MATRIX này để mô phỏng cách khuôn mặt 3D được chiếu lên mặt phẳng 2D của camera.
"""

MODEL_3D_POINTS = np.array([
    (0.0, 0.0, 0.0),            # Nose Tip (1)
    (-225.0, 170.0, -135.0),    # Left Eye (33)
    (225.0, 170.0, -135.0),     # Right Eye (263)
    (-150.0, -150.0, -125.0),   # Left Mouth (61)
    (150.0, -150.0, -125.0),    # Right Mouth (291)
    (0.0, -330.0, -65.0)        # Chin (152)
], dtype=np.float32)

#--- 3. Ngưỡng lọc dữ liệu bẩn ---
MAX_ACCEPTABLE_YAW = 45.0       # Độ quay ngang tối đa chấp nhận
MAX_ACCEPTABLE_PITCH = 45.0     # Độ gập tối đa chấp nhận

In [144]:
# 1. Hàm tính tỷ lệ khung hình mắt (EAR)
def eye_aspect_ratio(eye_coords):
    # eye_coords: 6 điểm mắt (p1-p6)
    # Lấy 6 điểm theo thứ tự (giả định đã đúng: p1-p4 là ngang, p2-p6 và p3-p5 là dọc)

    p1, p2, p3, p4, p5, p6 = eye_coords

    # Khoảng cách dọc:
    vertical_1 = dist.euclidean(p2, p6)
    vertical_2 = dist.euclidean(p3, p5)

    # Khoảng cách ngang:
    horizontal = dist.euclidean(p1, p4)

    # Công thức EAR:
    if horizontal == 0:
        return 0.001
    return (vertical_1 + vertical_2) / (2.0 * horizontal)

In [145]:
# 2. Hàm tính tỷ lệ khung hình miệng (MAR)
def mouth_aspect_ratio(mouth_coords):
    # mouth_coords: 4 điểm môi (61, 291, 0, 17)
    p1_h, p4_h, p2_v, p6_v = mouth_coords

    # Khoảng cách dọc
    vertical = dist.euclidean(p2_v, p6_v)
    # Khoảng cách ngan
    horizontal = dist.euclidean(p1_h, p4_h)

    if horizontal == 0:
        return 0.001
    return vertical / horizontal

In [146]:
#--- Hàm Tính góc quay đầu (Sử dụng PnP) ---
def get_head_pose(landmarks_list, w, h):
    """
    Tính góc quay đầu (Yaw, Pitch, Roll) dựa trên landmark 3D/2D.
    
    landmarks_list: danh sách landmark gốc (có Z) từ MediaPipe.
    w, h: Chiều rộng và chiều cao frame.
    """

    # 1. Trích xuất các điểm cần thiết (2D Image Points)
    image_points = []
    #landmarks_list: Giả định là danh sách các landmarks gốc của MP (Media Pipe)
    for i in HEAD_POSE_INDICES:
        # Sử dụng toạ độ chuẩn hoá (0-1) và nhân với kích thước ảnh (w, h)
        landmark = landmarks_list[i]
        x = int(landmark.x * w)
        y = int(landmark.y * h)
        image_points.append((x,y))
        # Mục đích: Giúp mô hình AI hoạt động ổn định và nhất quán trên các ảnh có độ phân giải khác nhau.
        """
        x = 0.5, y = 0.5: Luôn là điểm trung tâm của ảnh, bất kể ảnh lớn hay nhỏ.
        x = 1.0: Luôn là cạnh phải của ảnh 
        y = 0.0: Luôn là cạnh trên của ảnh (trong hầu hết các hệ tọa độ CV)
        """
    
    # Chuyển sang dạng Numpy array float 32 là định dạng bắt buộc cho hàm solvePnP của OpenCV.
    image_points = np.array(image_points, dtype=np.float32) 

    # 2. Thiết lập Ma trận Camera (Intrinsisic Matrix)
    # Camera Matrix giả định (dựa trên w, h)

    focal_length = 1 * w    # Tiêu cự f: Ước tính bằng chiều rộng của ảnh. Đây là một giả định chung khi không biết thông số camera chính xác.
    center = (w/2, h/2)     # Điểm trung tâm ảnh
    camera_matrix = np.array(
        [[focal_length, 0, center[0]],
         [0, focal_length, center[1]],
         [0, 0, 1]], dtype="double"
    ) # Xây dựng ma trận 3x3

    # Hệ số méo hình (Giả định = 0)
    dist_coeffs = np.zeros((4,1)) # Tạo ra một NumPy array có 4 hàng và 1 cột

    # 3. Giải bài toán PnP
    (success, rotation_vector, translation_vector) = cv2.solvePnP(
        MODEL_3D_POINTS, image_points, camera_matrix, dist_coeffs, flags = cv2.SOLVEPNP_ITERATIVE
    ) # Hàm cv2.solvePnP cố gắng tìm Ma trận Quay (rvec) và Ma trận Tịnh tiến (tvec) để khớp MODEL_3D_POINTS với image_points.

    # 4. Chuyển đổi Rotation Vector sang Góc Euler
    (rotation_matrix, jacobian) = cv2.Rodrigues(rotation_vector)
    """
    Chuyển Rotation Vector (rvec) thành Rotation Matrix} (3x3).
    Jacobian là ma trận phụ, không cần dùng.
    """

    #cv2.decomposeProjectionMatrix cho ra các gói Euler (Yaw, Pitch, Roll)
    proj_matrix = np.hstack((rotation_matrix, translation_vector))
    # Ghép Rotation Matrix và Translation Vector thành Projection Matrix (3x4)
    euler_angles = cv2.decomposeProjectionMatrix(proj_matrix)[6]
    #Hàm decomposeProjectionMatrix lấy ma trận P và phân tách nó thành các thành phần (trong đó, phần tử thứ 6 chứa các góc Euler).

    pitch = euler_angles[0,0]   # Góc ngửa / cuối đầu
    yaw = euler_angles[1,0]     # Góc quay đầu trái / phải
    roll = euler_angles[2,0]    # Góc nghiêng đầu sang vai

    return (yaw, pitch, roll), image_points

In [ ]:
#--- Hàm tính toán độ gục đầu (Slump) và nghiêng vai (Tilt)
def calculate_slump_features(face_landmarks, pose_landmarks, w, h):
    """Tính khoảng cách đầu-vai (Slump) và độ nghiêng vai (Tilt)."""

    d_slump = 0.0
    r_tilt = 0.0
    DUMMY_VALUE = -1.0

    # Kiểm tra tính sẵn có của landmark Pose (TH: Không thấy vai)
    if not pose_landmarks or len(pose_landmarks) < 13: # Cần ít nhất 13 điểm (đến vai)
        # Giả sử đầu nằm ở vị trí chuẩn nếu không tìm thấy pose
        d_slump = DUMMY_VALUE; r_tilt = DUMMY_VALUE
        return d_slump, r_tilt # -1.0, -1.0
    
    # Lấy điểm Vai và Mũi (sử dụng Pose Landmarker)
    p_left_shoulder = pose_landmarks[LEFT_SHOULDER_INDEX]
    p_right_shoulder = pose_landmarks[RIGHT_SHOULDER_INDEX]
    p_nose = pose_landmarks[NOSE_POSE_INDEX]

    # Chuyển đổi sang toạ độ pixel
    y_nose = p_nose.y * h
    y_shoulder_mid = (p_left_shoulder.y *h + p_right_shoulder.y * h) / 2

    # D_SLUMP (Khoảng cách dọc đầu so với vai)
    # Giá trị lớn / dương khi đầu gục xuống, nhỏ / âm khi đầu ngửa lên
    d_slump = y_nose - y_shoulder_mid # CẬP NHẬT CÔNG THỨC: Lấy Mũi - Vai để dương khi gục (mũi đi xuống)

    #R_TILT (Góc nghiêng vai)
    # Toạ độ pixel
    x_l_shoulder = p_left_shoulder.x * w
    y_l_shoulder = p_left_shoulder.y * h
    x_r_shoulder = p_right_shoulder.x * w
    y_r_shoulder = p_right_shoulder.y * h

    # Góc nghiêng so với dường ngang
    shoulder_angle_rad = np.arctan2(y_r_shoulder - y_l_shoulder, x_r_shoulder - x_l_shoulder)
    r_tilt = np.degrees(shoulder_angle_rad)

    return d_slump, r_tilt

In [ ]:
#--- Hàm tổ hợp, nơi logic lọc dữ liệu bẩn và logic phân loại Passed Out ---

# Giá trị Dummy / Place holder khi dữ liệu không tin cậy
DUMMY_VALUE = -1.0

def extract_features(image, face_landmarker, pose_landmarker):

    # 8 đặc trưng chính + 1 đặc trưng đánh dấu (UNRELIABLE_POSE) = 9
    features = np.zeros(9)
    h, w, c = image.shape

    # Chuyển BGR sang RGB và tạo đối tưởng Mediapipe Image
    rgb_image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    # SỬ DỤNG CONSTRUCTOR: Lấy ImageFormat từ alias 'mp_binding'
    # Lưu ý: Cú pháp cho RGB là 'RGB', không phải 'SRGB'
    mp_image = MpImage(image_format=mp.ImageFormat.SRGB, data=rgb_image)

    # 1. Phát hiện khuôn mặt
    face_result = face_landmarker.detect(mp_image)
    if not face_result.face_landmarks:
        # Nếu không tìm thấy, đặt UNRELIABLE_POSE = 1 (Giả định là Gục ngã nếu không thấy)
        features[:8] = DUMMY_VALUE # Gán -1.0 cho 8 đặc trưng đầu
        features[-1] = 1.0 #UNRELIABLE_POSE = 1.0
        return features, True # Vẫn chấp nhận frame để NN học MẪU "KHÔNG THẤY MẶT"
    
    face_landmarks = face_result.face_landmarks[0]

    # 2. Phát hiện tư thế (Pose)
    pose_result = pose_landmarker.detect(mp_image)
    pose_landmarks = pose_result.pose_landmarks[0] if pose_result.pose_landmarks else []

    # Trích xuất toạ độ Pixel (2D)
    landmarks_points = []
    for landmark in face_landmarks:
        x = int(landmark.x * w)
        y = int(landmark. y * h)
        landmarks_points.append((x,y))

    # --- A. TÍNH TOÁN TẤT CẢ CÁC ĐẶC TRƯNG ---
    # landmarks_list là list chứa đối tượng MP Landmark gốc (có Z)
    # 1. Head Pose (LUÔN TÍNH TOÁN)
    (yaw, pitch, roll), _ = get_head_pose(face_landmarks, w, h)

    # 2. EAR, MAR, Eye Closure (LUÔN TÍNH TOÁN)
    left_eye_coords = [landmarks_points[i] for i in LEFT_EYE_INDICES]
    right_eye_coords = [landmarks_points[i] for i in RIGHT_EYE_INDICES]
    mouth_coords = [landmarks_points[i] for i in MOUTH_INDICES]

    avg_ear = (eye_aspect_ratio(left_eye_coords) + eye_aspect_ratio(right_eye_coords)) / 2.0
    mar = mouth_aspect_ratio(mouth_coords)
    eye_closure = (dist.euclidean(left_eye_coords[1], left_eye_coords[5]) +
                   dist.euclidean(right_eye_coords[1], right_eye_coords[5])) / 2.0
    
    # 3. Slump, Tilt (LUÔN TÍNH TOÁN)
    d_slump, r_tilt = calculate_slump_features(face_landmarks, pose_landmarks, w, h)

    # --- B. Lọc độ tin cậy (UNRELIABLE_POSE) ---
    is_un_reliable_pose = 0.0
    
    # TH: Góc quay lớn (>45)
    if abs(yaw) > MAX_ACCEPTABLE_YAW or abs(pitch) > MAX_ACCEPTABLE_PITCH:
        is_unreliable_pose = 1.0 # Đánh dấu à không tin cậy/gục ngã

        # Gán DUMMY_VALUE cho các đặc trưng bị ảnh hưởng nặng bởi góc nhìn (EAR, MAR, Eye Closure)
        avg_ear = DUMMY_VALUE
        mar = DUMMY_VALUE
        eye_closure = DUMMY_VALUE

        # CHÚ Ý: GIỮ LẠI PITCH, YAW, ROLL, D_SLUMP, R_TILT
        # Các giá trị này (Pitch, Yaw, Roll, D_Slump, R_Tilt) 
        # CÓ THỂ là chính xác và là đặc trưng chính của hành vi Gục ngã.

    # Tổng hợp và trả về
    features = np.array([avg_ear, mar, pitch, yaw, roll, d_slump, r_tilt, eye_closure, is_unreliable_pose])

    # Luôn trả về True để cho phép NN học mẫu Passed Out
    return features, True


In [149]:
#--- Hàm main() và Khởi tạo Landmarker ---

def main():

    # Lấy đường dẫn tuyệt đối của thư mục làm việc hiện tại
    FACE_MODEL_PATH = "./model/face_landmarker.task"
    POSE_MODEL_PATH = "./model/pose_landmarker_full.task"

    # KIỂM TRA ĐƯỜNG DẪN CÓ TỒN TẠI KHÔNG
    if not os.path.exists(FACE_MODEL_PATH):
        print(f"FATAL ERROR: Cannot find file Face Landmarker at: {FACE_MODEL_PATH}")
        print("Please check the path again")
        return # Dừng chương trình nếu không tìm thấy
    
    if not os.path.exists(POSE_MODEL_PATH):
        print(f"FATAL ERROR: Cannot find file Pose Landmarker at: {POSE_MODEL_PATH}")
        print("Please check the path again")
        return # Dừng chương trình nếu không tìm thấy

    # Cấu hình Face Landmarker
    options_face = vision.FaceLandmarkerOptions(
        base_options = python.BaseOptions(model_asset_path = FACE_MODEL_PATH),
        num_faces = 1,
        running_mode = vision.RunningMode.IMAGE,
        min_face_detection_confidence = 0.5, # Thêm ngưỡng tin cậy
        min_tracking_confidence = 0.5
    )

    # Khởi tại Face Mesh
    face_landmarker = vision.FaceLandmarker.create_from_options(options_face)

    # Cấu hính Pose Landmarker
    options_pose = vision.PoseLandmarkerOptions(
        base_options = python.BaseOptions(model_asset_path = POSE_MODEL_PATH),
        running_mode = vision.RunningMode.IMAGE,
        min_pose_detection_confidence = 0.5,
        min_tracking_confidence = 0.5
    )
    
    # Khởi tạo Pose Landmarker 
    pose_landmarker = vision.PoseLandmarker.create_from_options(options_pose)

    data_rows = []

    # 2. Tạo thư mục Output (nếu chưa tồn tại)
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
        print(f"Output Folder created: {OUTPUT_DIR}")

    # 3. Lặp qua các thư mục (Labels)
    for label_index, label_name in enumerate(['awake', 'sleep', 'yawning', 'passed_out']):
        folder_path = os.path.join(DATA_ROOT, label_name)
        print(f"--- Processing folder: {label_name} ({label_index}) ---")

        # 4. Lặp qua các file ảnh
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(folder_path, filename)
                image = cv2.imread(image_path)

                if image is None: continue

                # 5. Trích xuất đặc trưng và đánh dấu độ tin cậy
                features, is_reliable = extract_features(image, face_landmarker, pose_landmarker)

                if is_reliable:
                    row = {
                        'EAR': features[0], 'MAR': features[1], 'PITCH': features[2],
                        'YAW': features[3], 'ROLL': features[4], 'D_SLUMP': features[5],
                        'R_TILT': features[6], 'EYE_CL': features[7],
                        'UNRELIABLE_POSE': features[8], # Cột mới
                        'Label': label_index,
                        'File_name': filename
                    }
                    data_rows.append(row)

    # 6. Xuất file .csv
    df = pd.DataFrame(data_rows)
    df.to_csv(OUTPUT_CSV, index = False)
    print(f"\n--- Hoàn tất! Đã lưu {len(data_rows)} dòng dữ liệu vào {OUTPUT_CSV} ---")

if __name__ == "__main__":
    main()


--- Processing folder: awake (0) ---
--- Processing folder: sleep (1) ---
--- Processing folder: yawning (2) ---
--- Processing folder: passed_out (3) ---

--- Hoàn tất! Đã lưu 2635 dòng dữ liệu vào ./Outputs\training-feature.csv ---


In [151]:
df = pd.read_csv('./Outputs/training-feature.csv', header = 0)
df

,EAR,MAR,PITCH,YAW,ROLL,D_SLUMP,R_TILT,EYE_CL,UNRELIABLE_POSE,Label,File_name
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0,image_0157_jpg.rf.65a32be073aa93f2bdf68abec695...
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0,image_0158_jpg.rf.d97ee16ebd2c38bfbe096de6025e...
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0,image_0160_jpg.rf.1e2b0d7c29c474e447ed8a1cf256...
3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0,image_0161_jpg.rf.a5d6b6ba01ac1cd8d0e479f2a886...
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0,image_0164_jpg.rf.18836600109b286e43fb4a72ae98...
...,...,...,...,...,...,...,...,...,...,...,...
2630,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,3,tired-woman-asleep-on-steering-wheel-in-her-ca...
2631,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,3,tired-young-woman-asleep-on-steering-wheel-in-...
2632,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,3,tired-young-woman-driver-asleep-on-pillow-on-s...
2633,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,3,tired-young-woman-sleep-in-car-hard-work-cause...
